In [2]:
import pandas as pd
import sklearn
import numpy as np
from prophet import Prophet

<h3 align='left'>Функция посылки</h3>

In [6]:
def kaggle_sub(y_pred):
    submission = pd.read_csv('sample_submission.csv')
    submission['sales'] = y_pred
    submission.to_csv('submission_prophet', index=False)

<h3 align='left'>Приводим заголовки в необходимый для Prophet вид</h3>

In [ ]:
test = pd.read_csv('test.csv', index_col='id').rename(columns={'date': 'ds', 'sales': 'y'})
train = pd.read_csv('train.csv', index_col='id').rename(columns={'date': 'ds', 'sales': 'y'})

In [ ]:
train['ds'] = pd.to_datetime(train['ds'])
test['ds'] = pd.to_datetime(test['ds'])

<h3 align='left'>Для каждого магазина и семейства товаров делаем предсказания в цикле</h3>

In [ ]:
store_numbers = train['store_nbr'].nunique()
store_families = train['family'].unique()
prediction_df = pd.DataFrame()
for i in range(1, store_numbers + 1):
    for j in store_families:
        train_store_ij = train[(train['store_nbr'] == i) & (train['family'] == j)][['ds', 'y']]
        pr = Prophet()
        pr.fit(train_store_ij)
        future = pr.make_future_dataframe(periods=16, include_history=False)
        predict = pr.predict(future)
        predict['store_nbr'] = i
        predict['family'] = j
        predict_df = pd.concat([predict_df, predict])



In [ ]:
prediction = test.reset_index().merge(prediction_df, on=['ds', 'store_nbr', 'family'])[['id', 'yhat']]\
                               .rename(columns={'yhat': 'sales'})

In [ ]:
prediction['sales'] = prediction['sales'].apply(lambda x: x if x>=0 else 0)

<h3 align='left'>Total rmsle-score 0.5152</h3>